In [0]:
import pandas as pd
import os

In [0]:
"""
Process Read Files

"""
root_path = '/content/'
orders = pd.read_csv(os.path.join(root_path,'orders.csv'))
products = pd.read_csv(os.path.join(root_path,'products.csv'))
users = pd.read_csv(os.path.join(root_path,'users.csv'))


In [5]:
#Here are all the users from the users table
users.head()

,user_id,user_name,location
0,1,Udit,Health Sciences
1,2,David,Port Jefferson
2,3,Kate,Port Jefferson
3,4,KJ,Port Jefferson
4,5,Jene,GNYC


In [0]:
# The query from the system: User with ID 2 is trying to buy something in the category "clothes" and has location as "Smithaven"
user_id = 2
user_category = 'clothes'
user_location = 'Smithaven'

In [0]:
#Find all the products that were bought by this user in the past by lookup in orders table.
products_bought_in_past = orders[orders['user_id'] == user_id]['product_id']

In [8]:
#Visualizing the list of products bought by this user in the past
products_bought_in_past.head()

12     8
22    14
Name: product_id, dtype: int64

In [0]:
#Find users that have bought the same products as the original user in the past.
related_users = orders[orders.product_id.isin(products_bought_in_past)]

In [10]:
#Visualizing list of related users and what orders they placed in the past
related_users.head()

,order_id,user_id,product_id
3,4,7,8
12,13,2,8
20,21,6,8
22,23,2,14
34,35,5,8


In [0]:
# Filter only the product information of the items these userId's have bought in the selected category
similar_products_from_other_users = orders[orders['user_id'].isin(related_users['user_id'])]['product_id']

In [12]:
#Visualizing product ids of products bought by matched users in the past.
similar_products_from_other_users.head()

2     4
3     8
6     6
12    8
13    7
Name: product_id, dtype: int64

In [0]:
#Joining products table with above table to find the complete information for the related products
recommended_products = pd.merge(products,similar_products_from_other_users,on='product_id')

In [14]:
#Visualizing products bought by users with same taste as orginal user.
recommended_products.head()

,product_id,product_name,store_location,store,discounts,price,shipping_time,category
0,1,LP Shirt,Smithaven,Smithaven Mall,20.0,10.0,3,clothes
1,2,Hard Drive,Smithaven,BestBuy,0.0,33.0,2,storage
2,4,Dell XPS 14,Smithaven,BestBuy,10.0,999.0,2,laptop
3,4,Dell XPS 14,Smithaven,BestBuy,10.0,999.0,2,laptop
4,6,Levis Jeans,Smithaven,Smithaven Mall,5.0,45.0,5,clothes


In [0]:
#Filter products with only the category that user queried for
recommended_products = recommended_products[recommended_products['category'] == user_category]

In [0]:
#Filter the products available in the user area
recommended_products = recommended_products[recommended_products['store_location'] == user_location]

In [17]:
recommended_products.head(10)

,product_id,product_name,store_location,store,discounts,price,shipping_time,category
0,1,LP Shirt,Smithaven,Smithaven Mall,20.0,10.0,3,clothes
4,6,Levis Jeans,Smithaven,Smithaven Mall,5.0,45.0,5,clothes
5,6,Levis Jeans,Smithaven,Smithaven Mall,5.0,45.0,5,clothes
6,7,Tommy Hilfiger T Shirt,Smithaven,Smithaven Mall,0.0,15.0,2,clothes
7,8,GAP Black Jacket,Smithaven,Smithaven Mall,0.0,70.0,3,clothes
8,8,GAP Black Jacket,Smithaven,Smithaven Mall,0.0,70.0,3,clothes
9,8,GAP Black Jacket,Smithaven,Smithaven Mall,0.0,70.0,3,clothes
10,8,GAP Black Jacket,Smithaven,Smithaven Mall,0.0,70.0,3,clothes


In [0]:
#Group same products together to see what products were bought the most as per their counts in the table
recommended_products_grouped = recommended_products.groupby('product_id').size().reset_index(name='counts')

In [0]:
#Fetching top 3 most sought after products in the area
recommended_products_grouped = recommended_products_grouped.nlargest(3,'counts')

In [20]:
#Visualizing top 3 most sought after products ids that will be recommended to the user.
recommended_products_grouped.head()

,product_id,counts
3,8,4
1,6,2
0,1,1


In [0]:
#Generating recommendations for this user
final_dataframe = products[products['product_id'].isin(recommended_products_grouped['product_id'])]

In [22]:
#Displaying all the receommended products to this user
final_dataframe.head()

,product_id,product_name,store_location,store,discounts,price,shipping_time,category
0,1,LP Shirt,Smithaven,Smithaven Mall,20.0,10.0,3,clothes
5,6,Levis Jeans,Smithaven,Smithaven Mall,5.0,45.0,5,clothes
7,8,GAP Black Jacket,Smithaven,Smithaven Mall,0.0,70.0,3,clothes
